In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import random, time, torchprofile
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Read and Transform Data
class LoadData(Dataset):
    def __init__(self, file, root):
        self.root = root
        
        # Data tranformation
        self.transform = transforms.Compose([
            transforms.Resize(84),
            transforms.CenterCrop(84),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomVerticalFlip(0.1),
            transforms.RandomPerspective(distortion_scale = 0.2, p = 0.2),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
        
        # Read the data file
        with open(file, 'r') as files:
            self.data = files.readlines()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        line = self.data[idx].strip()
        img_path, label = line.split()
        img_path = os.path.join(self.root, img_path)
        img = Image.open(img_path).convert('RGB')
        label = int(label)
        img = self.transform(img)
        return img, label

In [2]:
# Create dataset instances
train_dataset = LoadData(file = 'train.txt', root = '')
val_dataset = LoadData(file = 'val.txt', root = '')
test_dataset = LoadData(file = 'test.txt', root = '')

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

## Train handmade model

In [3]:
# Combine convolution, batch normalization, and ReLU into a module called dense block
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, kernel_size):
        super(DenseBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, growth_rate, kernel_size, padding = 1)
        self.norm = nn.BatchNorm2d(growth_rate)
        self.relu = nn.ReLU(inplace = True)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        return self.relu(x)

# Residual-in-Residual Dense Block (RRDB)
class RRDB(nn.Module):
    def __init__(self, in_channels, growth_rate = 32, kernel_size = 3, num_layers = 3):
        super(RRDB, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        channels = in_channels
        for i in range(num_layers):
            self.layers.append(DenseBlock(channels, growth_rate, kernel_size))
            channels += growth_rate

        self.conv = nn.Conv2d(channels, in_channels, kernel_size = 1, padding = 0)
        self.relu = nn.ReLU(inplace = True)

    def forward(self, x):
        out = x
        for layer in self.layers:
            new_out = layer(out)
            out = torch.cat([out, new_out], 1)
        out = self.conv(out)
        out = self.relu(out)
        return out + x

# Self-Attention Layer
class SelfAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, 1)
        self.scale = (in_channels // 8) ** -0.5

    def forward(self, x):
        batch_size, C, width, height = x.size()
        query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        key = self.key_conv(x).view(batch_size, -1, width * height)
        value = self.value_conv(x).view(batch_size, -1, height * width)
        scores = torch.bmm(query, key) * self.scale
        attention = F.softmax(scores, dim = -1)
        out = torch.bmm(value, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, width, height)
        return out + x

class SimpleCNN(nn.Module):
    def __init__(self, num_classes = 50):
        super(SimpleCNN, self).__init__()
        self.conv = DenseBlock(3, 512, 5)
        self.rrdb = RRDB(512)
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.attention = SelfAttention(512)
        self.pool2 = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = self.rrdb(x)
        x = self.pool1(x)
        x = self.attention(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Structure of model
model = SimpleCNN(num_classes = 50)
model.to(device)

SimpleCNN(
  (conv): DenseBlock(
    (conv): Conv2d(3, 512, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (norm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (rrdb): RRDB(
    (layers): ModuleList(
      (0): DenseBlock(
        (conv): Conv2d(512, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): DenseBlock(
        (conv): Conv2d(544, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (2): DenseBlock(
        (conv): Conv2d(576, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplac

In [4]:
epochs = 30
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr = lr, momentum = momentum, weight_decay = weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs * 0.5, epochs * 0.75, epochs * 0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Adjusted lr:' + str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total = len(train_loader), desc = "Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, label) in train_loader_iter:
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, label)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim = True)[1]
        train_acc += pred.eq(label.data.view_as(pred)).cpu().sum()
        loss.backward()
        
        optimizer.step()
        train_loader_iter.set_postfix(loss = loss.item(), accuracy = 100. * train_acc.item() / train_loader_len)    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))

def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total = len(val_loader), desc = "Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction = 'sum').item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss = test_loss / len(val_loader.dataset), accuracy = 100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))
    return accuracy

best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model.state_dict(), 'mycnn_best.pt')
        print('Best Accuracy: {:.2f}%'.format(best_val_acc))
print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [08:10<00:00,  2.02it/s, accuracy=9.57, loss=3.23] 


Train Epoch: 1, Loss: 3.572188, Accuracy: 9.57%


Validation Epoch #1: 100%|██████████| 8/8 [00:01<00:00,  4.49it/s, accuracy=12.9, loss=3.28] 


Validation Set: Average Loss: 3.2824, Accuracy: 12.89%
Best Accuracy: 12.89%


Training Epoch #2: 100%|██████████| 990/990 [08:10<00:00,  2.02it/s, accuracy=14.6, loss=3.18]


...

Validation Set: Average Loss: 1.9983, Accuracy: 44.22%


Training Epoch #30: 100%|██████████| 990/990 [08:11<00:00,  2.01it/s, accuracy=44.9, loss=1.72]


Train Epoch: 30, Loss: 1.938726, Accuracy: 44.88%


Validation Epoch #30: 100%|██████████| 8/8 [00:01<00:00,  4.60it/s, accuracy=45.8, loss=1.93] 

Validation Set: Average Loss: 1.9273, Accuracy: 45.78%
Best Accuracy: 45.78%
Final Best Accuracy: 45.78%


## Train ResNet34

In [5]:
model = models.resnet34(pretrained = False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.to(device)

/home/re6121029/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/re6121029/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
epochs = 30
lr = 0.01
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr = lr, momentum = momentum, weight_decay = weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs * 0.5, epochs * 0.75, epochs * 0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Adjusted lr:' + str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total = len(train_loader), desc = "Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, label) in train_loader_iter:
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, label)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim = True)[1]
        train_acc += pred.eq(label.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss = loss.item(), accuracy = 100. * train_acc.item() / train_loader_len)    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))

def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total = len(val_loader), desc = "Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction = 'sum').item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss = test_loss / len(val_loader.dataset), accuracy = 100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))
    return accuracy

best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model.state_dict(), 'resnet34_best.pt')
        print('Best Accuracy: {:.2f}%'.format(best_val_acc))
print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [02:53<00:00,  5.72it/s, accuracy=10.4, loss=3.15] 


Train Epoch: 1, Loss: 3.456496, Accuracy: 10.40%


Validation Epoch #1: 100%|██████████| 8/8 [00:01<00:00,  6.97it/s, accuracy=14.4, loss=3.15] 


Validation Set: Average Loss: 3.1548, Accuracy: 14.44%
Best Accuracy: 14.44%


Training Epoch #2: 100%|██████████| 990/990 [02:54<00:00,  5.69it/s, accuracy=19.5, loss=2.97]


...

Validation Set: Average Loss: 1.8484, Accuracy: 50.67%


Training Epoch #30: 100%|██████████| 990/990 [02:53<00:00,  5.69it/s, accuracy=76.9, loss=0.868]


Train Epoch: 30, Loss: 0.753628, Accuracy: 76.86%


Validation Epoch #30: 100%|██████████| 8/8 [00:01<00:00,  7.12it/s, accuracy=52.7, loss=1.87] 

Validation Set: Average Loss: 1.8698, Accuracy: 52.67%
Final Best Accuracy: 58.22%


## Test 2 models

In [12]:
mycnn_model = SimpleCNN(num_classes = 50)
mycnn_model.load_state_dict(torch.load("mycnn_best.pt"))
mycnn_model.to(device)
mycnn_model.eval()
resnet34_model = models.resnet34(pretrained = False)
resnet34_model.fc = torch.nn.Linear(model.fc.in_features, 50)
resnet34_model.load_state_dict(torch.load("resnet34_best.pt"))
resnet34_model.to(device)
resnet34_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
def test(model):
    model.eval()
    correct = 0
    test_dataset = LoadData(file = 'test.txt', root = '')
    test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total = len(test_loader), desc = "Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, label in test_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(label.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy = 100. * correct.item() / test_loader_len)
    

    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    

    # Calculate precision, recall, and F1-score
    precision = 100. * precision_score(all_targets, all_preds, average = 'macro')
    recall = 100. * recall_score(all_targets, all_preds, average = 'macro')
    f1 = 100. * f1_score(all_targets, all_preds, average = 'macro')
    
    # Calculate FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).to(device))
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

mycnn_acc, mycnn_precision, mycnn_recall, mycnn_f1, mycnn_flops, mycnn_elapsed_time = test(mycnn_model)
print(f"Accuracy: {mycnn_acc:.2f}%, Precision: {mycnn_precision:.4f}, Recall: {mycnn_recall:.4f}, F1 Score: {mycnn_f1:.4f}, FLOPS: {mycnn_flops:d}, Elapsed Time: {mycnn_elapsed_time:.2f} seconds")

resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(resnet34_model)
print(f"Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.4f}, Recall: {resnet_recall:.4f}, F1 Score: {resnet_f1:.4f}, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:01<00:00,  4.58it/s, accuracy=46.9]
/home/re6121029/.local/lib/python3.11/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


Accuracy: 46.89%, Precision: 51.6421, Recall: 46.8889, F1 Score: 46.7216, FLOPS: 7697151972, Elapsed Time: 1.75 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  7.22it/s, accuracy=55.1]


Accuracy: 55.11%, Precision: 55.8879, Recall: 55.1111, F1 Score: 54.2161, FLOPS: 612876800, Elapsed Time: 1.11 seconds
